## Исследование базы данных StackOverflow

### Данные
Самостоятельная работа основана на базе данных StackOverflow — сервиса вопросов и ответов о программировании.

Мы будем работать с версией базы, где хранятся данные о постах за 2008 год, но в таблицах есть информация и о более поздних оценках, которые эти посты получили.

### Цель
Проект состоит из двух частей:

В первой части несколько задач в SQL-тренажёре, чтобы закрепить пройденный материал.

Вторая часть проекта — аналитическая.

**ЧАСТЬ 1**

Задание 1

Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
''''
select count(p.id)
fromstackoverflow.posts p
left join stackoverflow.post_types pt ON p.post_type_id = pt.id
where pt.type = 'Question'
     and p.score > 300 or favorites_count >= 100 ;
''''     

Задание 2

Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
''''
select round(avg(quation_cnt))
 
 from 
 (select count(p.id) as quation_cnt,
       creation_date :: date as date_question
from   stackoverflow.posts as p
left join stackoverflow.post_types as pt
on p.post_type_id = pt.id
where pt.type = 'Question' and creation_date :: date between '2008-11-01' and '2008-11-18'
group by  creation_date :: date )  as tab
''''

Задание 3

Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
''''
select count(distinct u.id)       
from  stackoverflow.users u
join stackoverflow.badges b
on u.id = b.user_id
where u.creation_date :: date = b.creation_date :: date
''''

Задание 4

Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
''''
select count(t.id)
from (
     select p.id
     from stackoverflow.posts AS p
     join stackoverflow.votes AS v ON p.id = v.post_id
     join stackoverflow.users AS u ON p.user_id = u.id
     where u.display_name like 'Joel Coehoorn'
     group by p.id
     having count(v.id)>=1) as t;
''''     

Задание 5

Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
''''
select *,
       rank() over (order by id desc)
from  stackoverflow.vote_types 
order by id
''''

Задание 6

Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
''''
select u.id,
       count(vt.name) as voite_name
from  stackoverflow.users u
join stackoverflow.votes v 
on v.user_id = u.id
join stackoverflow.vote_types vt
on v.vote_type_id = vt.id
where vt.name = 'Close'
group by u.id
order by voite_name desc, u.id desc
limit 10
''''

Задание 7

Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно. Отобразите несколько полей: идентификатор пользователя; число значков; место в рейтинге — чем больше значков, тем выше рейтинг. Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге. Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
''''
select user_id,
      count(id),
      dense_rank() over (order by count(id) desc)
from stackoverflow.badges    
where creation_date:: date between '2008-11-15' and '2008-12-15'
group by user_id
order by count(id) desc,user_id asc
limit 10
''''


Задание 8

Сколько в среднем очков получает пост каждого пользователя?

Сформируйте таблицу из следующих полей:

заголовок поста;

идентификатор пользователя;

число очков поста;

среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
''''
select title,
       user_id,
       score,
     round (avg(score) over (partition by user_id ))
from   stackoverflow.posts   
where title is not null
      and score <> 0
group by   title, user_id, score 
''''

Задание 9

Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
''''
select title 
from  stackoverflow.posts
where user_id  in
                (select user_id
                 from stackoverflow.posts
                 group by user_id  
                 having count(id) > 1000)
and title is not null
group by title
''''

Задание 10

Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:

пользователям с числом просмотров больше либо равным 350 присвойте группу 1;

пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;

пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [ ]:
''''
select id,
       views,
       case
          when views >= 350 then 1
          when views >= 100 and views < 350  then 2
          else 3
      end 
from stackoverflow.users
where location like  '%United States%' and views != 0
''''

Задание 11

Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе.

Выведите поля с идентификатором пользователя, группой и количеством просмотров.

Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
''''
with tab AS
select t.id,
        t.views,
        t.group,
        max(t.views) over (partition by t.group) as max	
 from (select id,
              views,
              case
                 when views>=350 then 1
                 when views<100 then 3
                 else 2
              end as group
       from stackoverflow.users
       where location like '%United States%'
          and views > 0
          ) as t
  )
  
select tab.id, 
       tab.views,  
       tab.group
from tab
where tab.views = tab.max
order by tab.views desc, tab.id;
''''

Задание 12

Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:

- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [ ]:
''''
select day,
       count_id ,
       sum(count_id ) over (order by day)

from 
(select extract ( day from creation_date) as day,
        count(id)  as count_id        
from stackoverflow.users   
where  creation_date:: date between  '2008-11-01' and '2008-11-30'
group by extract ( day from creation_date)) as users_november
''''

Задание 13

Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста.

Отобразите:

- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [ ]:
''''
select distinct p.user_id,
       min(p.creation_date ) over (partition by p.user_id) - u.creation_date    
from stackoverflow.posts p
join stackoverflow.users u
on p.user_id = u.id
''''

ЧАСТЬ 2

Задание 1

Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить.

Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
''''
select date_trunc('month',creation_date)::date as month,
       sum(views_count) as sum_views
from  stackoverflow.posts
where date_trunc('month',creation_date)::date between '2008-01-01' and '2008-12-31'
group by month
order by sum_views desc
''''

Задание 2

Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов.

Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id.

Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
''''
select u.display_name,
       count(distinct p.user_id)
from stackoverflow.users u
join stackoverflow.posts p
on p.user_id = u.id
join stackoverflow.post_types pt
on p.post_type_id = pt.id
where p.creation_date::date between u.creation_date::date  and (u.creation_date::date + interval '1 month')
      and pt.type= 'Answer' 
group by u.display_name     
having count(distinct p.id)  > 100     
order by u.display_name
''''

Задание 3

Выведите количество постов за 2008 год по месяцам.

Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года.

Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
''''
select count(distinct id),
       DATE_TRUNC('month', creation_date)::date as month
from stackoverflow.posts
where user_id in
              (select u.id
               from stackoverflow.users u
               join stackoverflow.posts p
               on p.user_id = u.id
               where u.creation_date::date between '2008-09-01' and '2008-09-30'
               and p.creation_date:: date between '2008-12-01' and '2008-12-31'
               group by u.id  )    
group by DATE_TRUNC('month', creation_date)::date
order by month desc
''''

Задание 4

Используя данные о постах, выведите несколько полей:

- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумму просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
''''
select user_id,
       creation_date,
       views_count,
       sum(views_count) over (partition by user_id order by creation_date )
from  stackoverflow.posts 
order by user_id asc, creation_date asc
''''

Задание 5

Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой?

Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
''''
select round(avg(cnt_days))
from 
     (select user_id,
            count( creation_date :: date) as cnt_days
      from stackoverflow.posts
      where creation_date :: date between '2008-12-01' and '2008-12-07'
      group by user_id) as tab
''''

Задание 6

На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:

- номер месяца;
- количество постов за месяц;
- процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.


In [ ]:
''''
with tab as
       (select extract(month from creation_date) as month_number,
               count(id) as post_count
       from  stackoverflow.posts
       where creation_date::date between '2008-09-01' and '2008-12-31'
       group by month_number)

select *,
      round ((post_count :: numeric / lag(post_count) over (order by month_number) -1) *100,2)
from tab
''''

Задание 7

Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:

- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
''''
with t as (
         select user_id,
                count(distinct id) as cnt
         from stackoverflow.posts
         group by user_id
         orderby cnt desc
         limit 1),
         
t1 as (
       select p.user_id,
              p.creation_date,
              extract('week' from p.creation_date) AS week_number
       from stackoverflow.posts as p
       join t on t.user_id = p.user_id
       where DATE_TRUNC('month', p.creation_date)::date between  '2008-10-01' and '2008-10-30'
           )

select distinct week_number,
       max(creation_date) over (partition by week_number)
from t1
order by week_number;
''''